In [ ]:
import numpy as np
import SimpleITK as sitk
from beta_encoder import BetaEncoder
from classifier import R3DModel
import matplotlib.pyplot as plt
import torch

In [ ]:
encoder = BetaEncoder(in_ch=1, out_ch=5, base_ch=8, final_act='none')
decoder = R3DModel(num_classes=2)

In [3]:
img = sitk.ReadImage('data/preprocessed/final_haca3_ready.nii.gz')
thresholdFilter = sitk.OtsuThresholdImageFilter()
thresholdFilter.SetInsideValue(0)  # Mask 內部的像素值
thresholdFilter.SetOutsideValue(1) # Mask 外部的像素值
mask_sitk = thresholdFilter.Execute(img)
mask_sitk = sitk.GetArrayFromImage(mask_sitk)

In [4]:
img = sitk.GetArrayFromImage(img)
img.shape

(192, 224, 192)

In [6]:
encoder = encoder.eval()
decoder = decoder.eval()

encoder = encoder.to('cuda')
decoder = decoder.to('cuda')

In [7]:
batch_size = 1
slice_index = 50

axial_slice = img[slice_index, :, :]
axial_slice_mask = mask_sitk[slice_index, :, :]

axial_slice = axial_slice.astype(np.float32)
axial_tensor = torch.from_numpy(axial_slice)
axial_tensor = axial_tensor.unsqueeze(0).unsqueeze(0)

In [8]:
axial_tensor.shape
axial_slice_mask.shape

(224, 192)

In [9]:
output = encoder(axial_tensor.to('cuda')).detach().cpu().squeeze(0).numpy()

In [10]:
masked_output = output * axial_slice_mask